<a href="https://colab.research.google.com/github/bhagirathtallapragada/T-Natural-Language-Processing-CSC8980/blob/main/homework_3/tnlphw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bhagirath Tallapragada student id: 002575358

1. Using NLTK tokenize all documents, separated by polarity, remove stop words, and list
the top 20 most frequent tokens (and their counts) for the positive reviews, and the top
20 most frequent tokens (and their counts). What kind of things do you notice are
different between the two sets? (30 points)

In [157]:
import os
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
!tar -xvf /content/sample_data/review_polarity.tar.gz -C  /content/sample_data/

In [159]:
directory='/content/sample_data/txt_sentoken'
collection=[]
doc_name=[]
label=[]
for dir in os.listdir(directory):
  print(dir)
  if (dir == "neg"):
    dir1=directory+'/neg/'
    print(dir1)
    for filename in os.listdir(dir1):
      if (filename.endswith('.txt')):
        string1=(open(os.path.join(dir1, filename)))
        collection.append(string1.read())
        doc_name.append(filename.rstrip('.txt'))
        label.append(0)
  elif (dir == "pos"):
    dir1=directory+'/pos/'
    print(dir1)
    for filename in os.listdir(dir1):
      if (filename.endswith('.txt')):
        string1=(open(os.path.join(dir1, filename)))
        collection.append(string1.read())
        doc_name.append(filename.rstrip('.txt'))
        label.append(1)

pos
/content/sample_data/txt_sentoken/pos/
neg
/content/sample_data/txt_sentoken/neg/


In [160]:
print(len(label))

2000


In [161]:
tokens=[]
for i in collection:
      tokens.append(word_tokenize(i.rstrip('\n')))

In [162]:
collection[0]

'there seem to be two reactions to dark city . \nyou either love it or you are unimpressed . \ncount me as a " love it . " \nas good as the crow and spawn are , dark city is better . \nwhere the former two have a specific comic-book following , the latter is an original work with the potential for broader appeal . \nit has a similar look to the others , but dark city is much more thoughtful and much less violent . \nnot clever or complex , the story is still fascinating , like a selection from the twilight zone . \na man ( rufus sewell ) is awakened in his bathtub by a man with a syringe . \nsurprised , the attacker runs off . \nthe dazed man gets out of the tub and finds a woman\'s body in the room . \na mysterious caller warns him to leave the apartment immediately , so he packs some clothes and leaves , never sure why he\'s leaving or what he\'s fleeing . \nghoulish figures in long coats recognize the man and chase him through the dark city , hounding him like figures in a nightmare

In [163]:
training_set=pd.DataFrame(list(zip(doc_name, collection, tokens, label)), columns =['doc_name', 'text', 'tokens', 'label'])

In [164]:
training_set.head()

,doc_name,text,tokens,label
0,cv684_11798,there seem to be two reactions to dark city . ...,"[there, seem, to, be, two, reactions, to, dark...",1
1,cv990_11591,the relaxed dude rides a roller coaster \nthe ...,"[the, relaxed, dude, rides, a, roller, coaster...",1
2,cv187_12829,weir is well-respected in hollywood for turnin...,"[weir, is, well-respected, in, hollywood, for,...",1
3,cv441_13711,"call me crazy , but i don't see saving private...","[call, me, crazy, ,, but, i, do, n't, see, sav...",1
4,cv967_5788,"is it just me , or have disney films gradually...","[is, it, just, me, ,, or, have, disney, films,...",1


In [165]:
stops = stopwords.words('english')
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [166]:
print(training_set.iloc[0]['tokens'])

['there', 'seem', 'to', 'be', 'two', 'reactions', 'to', 'dark', 'city', '.', 'you', 'either', 'love', 'it', 'or', 'you', 'are', 'unimpressed', '.', 'count', 'me', 'as', 'a', '``', 'love', 'it', '.', '``', 'as', 'good', 'as', 'the', 'crow', 'and', 'spawn', 'are', ',', 'dark', 'city', 'is', 'better', '.', 'where', 'the', 'former', 'two', 'have', 'a', 'specific', 'comic-book', 'following', ',', 'the', 'latter', 'is', 'an', 'original', 'work', 'with', 'the', 'potential', 'for', 'broader', 'appeal', '.', 'it', 'has', 'a', 'similar', 'look', 'to', 'the', 'others', ',', 'but', 'dark', 'city', 'is', 'much', 'more', 'thoughtful', 'and', 'much', 'less', 'violent', '.', 'not', 'clever', 'or', 'complex', ',', 'the', 'story', 'is', 'still', 'fascinating', ',', 'like', 'a', 'selection', 'from', 'the', 'twilight', 'zone', '.', 'a', 'man', '(', 'rufus', 'sewell', ')', 'is', 'awakened', 'in', 'his', 'bathtub', 'by', 'a', 'man', 'with', 'a', 'syringe', '.', 'surprised', ',', 'the', 'attacker', 'runs', '

In [167]:
# Function to remove stop words
# def remove_sw(li1, li2):
#     return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

def remove_sw(l1,stop):
  return [w for w in l1 if not w in stop] 

ind=0
for i in training_set['tokens']: 
  x=remove_sw(i,stops)
  # print(x)
  training_set.at[ind,'tokens']=x
  ind+=1

In [168]:
training_set.head()
# training_set['tokens'][0]

,doc_name,text,tokens,label
0,cv684_11798,there seem to be two reactions to dark city . ...,"[seem, two, reactions, dark, city, ., either, ...",1
1,cv990_11591,the relaxed dude rides a roller coaster \nthe ...,"[relaxed, dude, rides, roller, coaster, big, l...",1
2,cv187_12829,weir is well-respected in hollywood for turnin...,"[weir, well-respected, hollywood, turning, scr...",1
3,cv441_13711,"call me crazy , but i don't see saving private...","[call, crazy, ,, n't, see, saving, private, ry...",1
4,cv967_5788,"is it just me , or have disney films gradually...","[,, disney, films, gradually, lost, appeal, ?,...",1


In [169]:
# computing frequency of words for each label:
p=training_set['label'].value_counts()[0]
n=training_set['label'].value_counts()[1]

pos_wrds=[training_set.loc[training_set['label'] == 1, 'tokens'].iloc[i] for i in range(p)]
neg_wrds=[training_set.loc[training_set['label'] == 0, 'tokens'].iloc[i] for i in range(n)]

In [170]:
print(neg_wrds[0])
# print(pos_wrds)

['attempt', 'florida', 'film', 'noir', ',', 'palmetto', 'fails', 'fundamental', 'levels', '.', "'s", 'slow', 'moving', ',', 'uninvolving', ',', 'plain', 'uninteresting', '.', 'harry', 'barber', '(', 'woody', 'harrelson', ')', 'ex-newspaper', 'man', ',', 'jail', 'framed', 'crime', '.', 'luck', "n't", 'best', ',', 'things', 'looking', 'mysterious', 'woman', ',', 'rhea', 'malroux', '(', 'elisabeth', 'shue', ')', ',', 'approaches', 'proposition', '.', 'wants', 'harry', 'help', 'stepdaughter', ',', 'odette', '(', 'chloe', 'sevigny', ')', ',', 'pull', 'fake', 'kidnapping', 'scheme', 'get', '$', '500', ',', '000', 'stingy', ',', 'rich', ',', 'husband', 'felix', '(', 'rolf', 'hoppe', ')', '.', 'first', ',', 'harry', 'expected', 'provide', 'threatening', 'voice', 'phone', ',', 'collect', 'money', '(', 'gets', 'keep', '10', '%', ')', '.', ',', 'deed', 'carried', ',', 'things', 'seem', ',', 'harry', 'gets', 'caught', 'ensuing', 'storm', '.', 'palmetto', 'pulls', 'stops', 'achieve', 'film', 'noir'

In [171]:
pos_wrds=list(pos_wrds)
neg_wrds=list(neg_wrds)

print(len(pos_wrds))
print(len(neg_wrds))

pw=[]
nw=[]
for i in pos_wrds:
  pw.extend(i)
for i in neg_wrds:
  nw.extend(i)

print(len(pw))
print(len(nw))

1000
1000
489068
440140


In [172]:
pw_count={}
nw_count={}

for wrd in pw:
  if wrd in pw_count:
    pw_count[wrd]+=1
  else:
    pw_count[wrd]=1

for wrd in nw:
  if wrd in nw_count:
    nw_count[wrd]+=1
  else:
    nw_count[wrd]=1

In [ ]:
for key, value in pw_count.items(): 
        print ("% s : % d"%(key, value))
for key, value in nw_count.items(): 
        print ("% s : % d"%(key, value))

In [174]:
pwc_sorted={k: v for k, v in sorted(pw_count.items(), key=lambda item: item[1], reverse=True)}.items()
nwc_sorted={k: v for k, v in sorted(nw_count.items(), key=lambda item: item[1], reverse=True)}.items()

In [175]:
print('Positive words')
print(list(pwc_sorted)[:20])
print('negative words')
print(list(nwc_sorted)[:20])

Positive words
[(',', 42448), ('.', 33714), ("'s", 9473), ('``', 8494), (')', 6039), ('(', 6014), ('film', 5186), ('one', 2943), ("n't", 2775), ('movie', 2497), ('like', 1713), ('?', 1570), (':', 1502), ('story', 1231), ('also', 1200), ('good', 1190), ('even', 1175), ('time', 1171), ('would', 1079), ('character', 1067)]
negative words
[(',', 35269), ('.', 32162), ('``', 9123), ("'s", 8655), (')', 5742), ('(', 5650), ('film', 4257), ("n't", 3442), ('movie', 3174), ('one', 2637), ('?', 2201), ('like', 1832), (':', 1540), ('even', 1381), ('would', 1185), ('good', 1126), ('time', 1111), ('!', 1056), ('get', 1039), ('bad', 1019)]


The following are the main differences observed in the two lists:

a. The negative list consists of higher frequency for expressions that pertain to negative connotations, obvious ones being words like 'bad'.

b. Another interesting observation is the existence of similar elements in the two sets. For instance the expression "n't" has presence in both the lists, however the frequency of this expression is much higher in the negative context (possibly found in words that express negative expectation such as "isn't", "shouldn't", "doesn't").

c. The above observation also points out ambiguity. words with literal positive meanings do not necessarily occur only in positive contexts. For instance, "like" is a term that occurs more in the negative set inspite of its general inference being associated with positivity.

The above points argue for the fact that context is a very important aspect that should not be ignored. Eliminating the stop words does help to identify the important words yet it does not gaurantee context preservation.

2. Using the code from previous lectures, build 3 polarity classifiers using the following
parameters (20 points). Note: just train the models.
a) For training: use 50% of the positive dataset and 70% of the negative dataset. For
your model use: NaiveBayes with the TF-IDF vectorizer.
b) For training: use 50% of the negative dataset and 70% of the positive dataset. For
your model use: NaiveBayes with the TF-IDF vectorizer.
c) For training: use 25% of the negative dataset and 25% of the positive dataset. For
your model use: SVM with the TF-IDF vectorizer

In [176]:
# Function to return the trained model

def model_trainer(algo, vm, training_set):
  model = make_pipeline(vm, algo)
  model.fit(training_set['text'],training_set['label'])

  return model

In [177]:
# a. For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

p_train1,p_test1=train_test_split(training_set.query('label==1'), test_size=0.50, random_state=12345)
n_train1,n_test1=train_test_split(training_set.query('label==0'), test_size=0.30, random_state=12345)

print(len(p_train1))
print(len(n_train1))
train_data1=pd.concat([p_train1,n_train1])
test_data1=pd.concat([p_test1,n_test1])
train_data1.shape

500
700


(1200, 4)

In [178]:
# a. Naive Bayes, TFIDF
model_a_tf=model_trainer(MultinomialNB(),TfidfVectorizer(), train_data1)

In [179]:
# b. For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

p_train2,p_test2=train_test_split(training_set.query('label==1'), test_size=0.30, random_state=12345)
n_train2,n_test2=train_test_split(training_set.query('label==0'), test_size=0.50, random_state=12345)

print(len(p_train2))
print(len(n_train2))
train_data2=pd.concat([p_train2,n_train2])
test_data2=pd.concat([p_test2,n_test2])
train_data2.shape

700
500


(1200, 4)

In [180]:
model_b_tf=model_trainer(MultinomialNB(),TfidfVectorizer(), train_data2)

In [181]:
# c. For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer.

p_train3,p_test3=train_test_split(training_set.query('label==1'), test_size=0.75, random_state=12345)
n_train3,n_test3=train_test_split(training_set.query('label==0'), test_size=0.75, random_state=12345)

print(len(p_train3))
print(len(n_train3))
train_data3=pd.concat([p_train3,n_train3])
test_data3=pd.concat([p_test3,n_test3])
train_data3.shape

250
250


(500, 4)

In [182]:
model_c_tf=model_trainer(SVC(),TfidfVectorizer(), train_data3)

3. Using the models from question 2, evaluate them on their individual rest of the dataset.
This is, for a) 50% positive and 30% negative, for b) 50% negative and 30% positive, and
for c) 75% negative and 75% positive. Calculate and show ONLY the following metrics for
each model: Accuracy, Precision, Recall, Macro F1-score. (15 points).

ans. Training and Test data separated to required ratio in question two itself. Using the test data now for evaluation. 

In [183]:
labels_a_tf = model_a_tf.predict(test_data1['text'])

In [184]:
print('*** Evaluation of Model a ***')
print('Accuracy:', sklearn.metrics.accuracy_score(test_data1['label'],labels_a_tf))
print('Precision:', sklearn.metrics.precision_score(test_data1['label'],labels_a_tf))
print('Recall:', sklearn.metrics.recall_score(test_data1['label'],labels_a_tf))
print('F1 Score:', sklearn.metrics.f1_score(labels_a_tf, test_data1['label'], average='macro'))
print('***')

*** Evaluation of Model a ***
Accuracy: 0.38
Precision: 1.0
Recall: 0.008
F1 Score: 0.28165913567373424
***


In [185]:
labels_b_tf = model_b_tf.predict(test_data2['text'])

In [186]:
print('*** Evaluation of Model b ***')
print('Accuracy:', sklearn.metrics.accuracy_score(test_data2['label'],labels_b_tf))
print('Precision:', sklearn.metrics.precision_score(test_data2['label'],labels_b_tf))
print('Recall:', sklearn.metrics.recall_score(test_data2['label'],labels_b_tf))
print('F1 Score:', sklearn.metrics.f1_score(labels_b_tf, test_data2['label'], average='macro'))
print('***')

*** Evaluation of Model b ***
Accuracy: 0.375
Precision: 0.375
Recall: 1.0
F1 Score: 0.2727272727272727
***


In [187]:
labels_c_tf = model_c_tf.predict(test_data3['text'])

In [188]:
print('*** Evaluation of Model c ***')
print('Accuracy:', sklearn.metrics.accuracy_score(test_data3['label'],labels_c_tf))
print('Precision:', sklearn.metrics.precision_score(test_data3['label'],labels_c_tf))
print('Recall:', sklearn.metrics.recall_score(test_data3['label'],labels_c_tf))
print('F1 Score:', sklearn.metrics.f1_score(labels_c_tf, test_data3['label'], average='macro'))
print('***')

*** Evaluation of Model c ***
Accuracy: 0.7533333333333333
Precision: 0.7480417754569191
Recall: 0.764
F1 Score: 0.7533052649545904
***


4) Using the model performance metrics from question 3, answer the following
questions. Please provide logical and intuitive rationale for your answers, simple
answers like: because it has the best score, will not be sufficient. (40 points):
a) What is the best performing model?
b) Why do you think this is the best performing model?
c) How does class imbalance play in determining polarity?
d) Do you think either more data or a better model is a better approach for this
kind of task?

Ans. a. The best performing model here is the SVM model (model c).

b. Of all the models, the SVM model was fed an equal proportion of data (25% positive and 25% negative). Therefore, the SVM model's input is the least biased of all at the outset.
Another factor is that the model achieves ~74% precision and a ~76% recall with lesser training data. This can be attributed to the fact that SVM does a better job at protecting the context unlike the Naive Bayes method where features are assumed independent.

c. Class imbalance can play a significant role in determining polarity as it biases the model towards one class over the other. It can result in misclassification. For instance if a model is extensively trained to handle phrases with negative connotation such as 'would'nt', 'definitely not', there is an increased chance of the model weighing these words negatively even in case of relatively positive connotations, for example: 'I wouldn't say it is perfect yet it manages to keep my attention ...' or 'this deep film is definitely not for featherheads'.

d. I think having extensive data is not nocessarily a boon for this task. This is due to the immense ambiguity present in generic user reviews. As humans we can associate irony, sarcasm, amazement and shock to various contexts (inclusive of opposite implications). Since the algorithm will not be equipped with such intuition, it is easy for it to get polarized by immense data that is innately biased towards a certain style of expressing.
A better model on the other hand can work with lesser data but exposed to wider variations in the dataset and therefore still do better job of classification.

5) Using NLTK and VADER, calculate the sentiment score for all documents in the
positive polarity. Calculate the polarity threshold needed (and reasonable) to have the
majority of the document labels match. Do the same for the negative class. Provide the
threshold needed, the reason why you think this threshold is reasonable, and the
accuracy percentage (how many documents are correctly labeled using this threshold).
(45 points):

In [189]:
pos_sent_scores=[]
sia = SIA()
# pia_dict={}

ind=0
for i in training_set.query('label==1')['text']:
  pos_sent_scores.append(sia.polarity_scores(i))
  ind+=1

pos_sent_scores

[{'compound': 0.9956, 'neg': 0.078, 'neu': 0.761, 'pos': 0.162},
 {'compound': 0.9955, 'neg': 0.064, 'neu': 0.827, 'pos': 0.109},
 {'compound': 0.9957, 'neg': 0.062, 'neu': 0.764, 'pos': 0.174},
 {'compound': 0.8974, 'neg': 0.119, 'neu': 0.749, 'pos': 0.132},
 {'compound': 0.9997, 'neg': 0.059, 'neu': 0.734, 'pos': 0.207},
 {'compound': 0.9994, 'neg': 0.096, 'neu': 0.706, 'pos': 0.199},
 {'compound': 0.9835, 'neg': 0.102, 'neu': 0.765, 'pos': 0.133},
 {'compound': 0.9412, 'neg': 0.105, 'neu': 0.777, 'pos': 0.119},
 {'compound': 0.9883, 'neg': 0.075, 'neu': 0.746, 'pos': 0.179},
 {'compound': 0.9971, 'neg': 0.087, 'neu': 0.729, 'pos': 0.184},
 {'compound': 0.9993, 'neg': 0.079, 'neu': 0.716, 'pos': 0.205},
 {'compound': -0.9181, 'neg': 0.12, 'neu': 0.771, 'pos': 0.109},
 {'compound': 0.9973, 'neg': 0.039, 'neu': 0.802, 'pos': 0.159},
 {'compound': 0.9808, 'neg': 0.144, 'neu': 0.68, 'pos': 0.177},
 {'compound': -0.8146, 'neg': 0.146, 'neu': 0.72, 'pos': 0.134},
 {'compound': 0.998, 'neg'

Considering the compound polarity scores, we can see that the spectrum of the scores lies between -0.99 to +0.99. A majority of the samples lies in the interval (0.8 to 1) for positive labeled data. A minor section of the data is misclassified with negative scores. An even smaller set of samples lie in the region of (0.2 to 0.6).

In order to ensure that the majority of the data is labelled correctly it is required that we consider the lowest possible score that implies positive classification by Vader. In the above data such score is ~ 0.23. therefore picking this value as threshold will ensure that the majority samples are correctly labeled while also ensuring that outliers that have ambiguous scores yet do not convey negative classification are protected.

In [210]:
# Calculating the accuracy percentage of using threshold of 0.23
threshold=0.23
pos_val=[]
neg_val=[]
for i in pos_sent_scores:
  if i['compound']>threshold:
    pos_val.append(i)
  else:
    neg_val.append(i)


In [215]:
print(len(pos_val))
print(len(neg_val))

# Accuracy percentage
p_accuracy=(len(pos_val)/len(pos_sent_scores))*100
print("Accuracy percentage: {}".format(p_accuracy))

824
176
Accuracy percentage: 82.39999999999999


In [190]:
# Negative sentiment scores

neg_sent_scores=[]
sia = SIA()


ind=0
for i in training_set.query('label==0')['text']:
  neg_sent_scores.append(sia.polarity_scores(i))
  ind+=1

neg_sent_scores

[{'compound': -0.5308, 'neg': 0.102, 'neu': 0.806, 'pos': 0.092},
 {'compound': -0.9941, 'neg': 0.153, 'neu': 0.73, 'pos': 0.116},
 {'compound': -0.7614, 'neg': 0.096, 'neu': 0.82, 'pos': 0.084},
 {'compound': -0.93, 'neg': 0.119, 'neu': 0.794, 'pos': 0.087},
 {'compound': -0.9926, 'neg': 0.184, 'neu': 0.726, 'pos': 0.089},
 {'compound': 0.9965, 'neg': 0.149, 'neu': 0.645, 'pos': 0.206},
 {'compound': -0.9642, 'neg': 0.118, 'neu': 0.828, 'pos': 0.054},
 {'compound': -0.983, 'neg': 0.201, 'neu': 0.648, 'pos': 0.152},
 {'compound': -0.743, 'neg': 0.138, 'neu': 0.74, 'pos': 0.122},
 {'compound': 0.9808, 'neg': 0.101, 'neu': 0.757, 'pos': 0.142},
 {'compound': 0.836, 'neg': 0.137, 'neu': 0.703, 'pos': 0.161},
 {'compound': 0.7391, 'neg': 0.015, 'neu': 0.915, 'pos': 0.069},
 {'compound': 0.6443, 'neg': 0.116, 'neu': 0.769, 'pos': 0.116},
 {'compound': 0.9883, 'neg': 0.105, 'neu': 0.752, 'pos': 0.143},
 {'compound': 0.9245, 'neg': 0.104, 'neu': 0.773, 'pos': 0.123},
 {'compound': 0.2982, 'ne

In this case we can see that inspite of the samples being from negative class, less than 25% of the samples have negative polarity score. While over 50% of the data has a score of over +0.5.

Applying similar inference as the positive case and analysing the spectrum of scores, we can deduce that the threshold value that allows similar classification as our labels is ~ 0.95. Therefore using this as threshold helps us to accurately classify a majority of the negative samples as negative.

In [242]:
# Calculating the accuracy percentage of using threshold of 0.23
threshold=0.95
pos_val1=[]
neg_val1=[]
for i in neg_sent_scores:
  if i['compound']<threshold:
    neg_val1.append(i)
  else:
    pos_val1.append(i)


In [243]:
print(len(pos_val1))
print(len(neg_val1))

# Accuracy percentage
print(len(neg_sent_scores))
n_accuracy=(len(neg_val1)/len(neg_sent_scores))*100
print("Accuracy percentage: {}".format(n_accuracy))

359
641
1000
Accuracy percentage: 64.1


Bonus (40 points): Repeat questions 2,3 and 4 removing all stopwords. Answer the
following questions: Did this change the results in any way? Why do you think so?

In [191]:
# adding a column to training_set to stringify the tokens after removing stop words
training_set.insert(3,'strminstop','')

In [192]:
ind=0
for i in training_set['tokens']:
  str1 = ' '.join([str(elem) for elem in i])
  training_set.at[ind,'strminstop']=str1
  ind+=1

In [193]:
training_set.head()

,doc_name,text,tokens,strminstop,label
0,cv684_11798,there seem to be two reactions to dark city . ...,"[seem, two, reactions, dark, city, ., either, ...",seem two reactions dark city . either love uni...,1
1,cv990_11591,the relaxed dude rides a roller coaster \nthe ...,"[relaxed, dude, rides, roller, coaster, big, l...",relaxed dude rides roller coaster big lebowski...,1
2,cv187_12829,weir is well-respected in hollywood for turnin...,"[weir, well-respected, hollywood, turning, scr...",weir well-respected hollywood turning scripts ...,1
3,cv441_13711,"call me crazy , but i don't see saving private...","[call, crazy, ,, n't, see, saving, private, ry...","call crazy , n't see saving private ryan film ...",1
4,cv967_5788,"is it just me , or have disney films gradually...","[,, disney, films, gradually, lost, appeal, ?,...",", disney films gradually lost appeal ? almost ...",1


In [194]:
# Function to return the trained model tweaked to consider the column that has strings with stop words eliminated

def model_trainer1(algo, vm, training_set):
  model = make_pipeline(vm, algo)
  model.fit(training_set['strminstop'],training_set['label'])

  return model

In [195]:
# a. For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

bp_train1,bp_test1=train_test_split(training_set.query('label==1'), test_size=0.50, random_state=12345)
bn_train1,bn_test1=train_test_split(training_set.query('label==0'), test_size=0.30, random_state=12345)

print(len(bp_train1))
print(len(bn_train1))
btrain_data1=pd.concat([bp_train1,bn_train1])
btest_data1=pd.concat([bp_test1,bn_test1])
btrain_data1.shape

500
700


(1200, 5)

In [196]:
# a. Naive Bayes, TFIDF
bmodel_a_tf=model_trainer1(MultinomialNB(),TfidfVectorizer(), btrain_data1)

In [197]:
# b. For training: use 50% of the negative dataset and 70% of the positive dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.

bp_train2,bp_test2=train_test_split(training_set.query('label==1'), test_size=0.30, random_state=12345)
bn_train2,bn_test2=train_test_split(training_set.query('label==0'), test_size=0.50, random_state=12345)

print(len(bp_train2))
print(len(bn_train2))
btrain_data2=pd.concat([bp_train2,bn_train2])
btest_data2=pd.concat([bp_test2,bn_test2])
btrain_data2.shape

700
500


(1200, 5)

In [198]:
bmodel_b_tf=model_trainer1(MultinomialNB(),TfidfVectorizer(), btrain_data2)

In [244]:
# c. For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer.

bp_train3,bp_test3=train_test_split(training_set.query('label==1'), test_size=0.75, random_state=12345)
bn_train3,bn_test3=train_test_split(training_set.query('label==0'), test_size=0.75, random_state=12345)

print(len(bp_train3))
print(len(bn_train3))
btrain_data3=pd.concat([bp_train3,bn_train3])
btest_data3=pd.concat([bp_test3,bn_test3])
btrain_data3.shape

250
250


(500, 5)

In [245]:
bmodel_c_tf=model_trainer1(SVC(),TfidfVectorizer(), btrain_data3)

In [201]:
blabels_a_tf = bmodel_a_tf.predict(btest_data1['strminstop'])

In [202]:
print('*** Evaluation of Model a ***')
print('Accuracy:', sklearn.metrics.accuracy_score(btest_data1['label'],blabels_a_tf))
print('Precision:', sklearn.metrics.precision_score(btest_data1['label'],blabels_a_tf))
print('Recall:', sklearn.metrics.recall_score(btest_data1['label'],blabels_a_tf))
print('F1 Score:', sklearn.metrics.f1_score(blabels_a_tf, btest_data1['label'], average='macro'))
print('***')

*** Evaluation of Model a ***
Accuracy: 0.39625
Precision: 1.0
Recall: 0.034
F1 Score: 0.3098903218547233
***


In [246]:
blabels_b_tf = bmodel_b_tf.predict(btest_data2['strminstop'])

In [250]:
print('*** Evaluation of Model b ***')
print('Accuracy:', sklearn.metrics.accuracy_score(btest_data2['label'],blabels_b_tf))
print('Precision:', sklearn.metrics.precision_score(btest_data2['label'],blabels_b_tf))
print('Recall:', sklearn.metrics.recall_score(btest_data2['label'],blabels_b_tf))
print('F1 Score:', sklearn.metrics.f1_score(blabels_b_tf, btest_data2['label'], average='macro'))
print('***')

*** Evaluation of Model b ***
Accuracy: 0.38375
Precision: 0.37831021437578816
Recall: 1.0
F1 Score: 0.28828063109152563
***


In [248]:
blabels_c_tf = bmodel_c_tf.predict(btest_data3['strminstop'])

In [251]:
print('*** Evaluation of Model c ***')
print('Accuracy:', sklearn.metrics.accuracy_score(btest_data3['label'],blabels_c_tf))
print('Precision:', sklearn.metrics.precision_score(btest_data3['label'],blabels_c_tf))
print('Recall:', sklearn.metrics.recall_score(btest_data3['label'],blabels_c_tf))
print('F1 Score:', sklearn.metrics.f1_score(blabels_c_tf, btest_data3['label'], average='macro'))
print('***')

*** Evaluation of Model c ***
Accuracy: 0.788
Precision: 0.7706766917293233
Recall: 0.82
F1 Score: 0.7877826894740214
***


Ans. a. The best performing model continues to be SVM model (model c).

b. The model maintains neutrality. However the recall rate of 0.82 suggests that it manages little improvement upon eliminating the stop words. This conveys that the context lost due to elimination of stop words has a limiting impact on SVM.  Naive Bayes on the other hand has improved in both cases when compared to Q2 and 3 scenario since the model considers features to be independent of each other, hence the elimination of stop words works in favor of Naive Bayes.

c. The views about class imbalance remain the same as question 4, however this example conveys scope for improvement for certain models inspite of biased input.


d. I still think having extensive data is not nocessarily a boon for this task. This is a good example of a better model performing with lower data, simply because of its suitability (less dependence on stop words).

The elimination of stop words and training the data without the stop words did influence the results by a small margin. We can see that the precision and recall values of the SVM model did not improve much from the earlier model trained on the actual text and the overall analysis pointing to an improvement in the Naive Bayes model (model b) due to reasons explained in the above text cell.